In [ ]:
from pathlib import Path
import pandas as pd
from spacy.scorer import PRFScore

from collections import Counter

In [ ]:
exp_dir = Path("dev_dataset_split_by_names/")

train_true_df_path = exp_dir / "cyber_attrs_training_df.zip"
train_pred_df_path = exp_dir / "cyber_attrs_training_by_model_df.zip"
test_true_df_path = exp_dir / "cyber_attrs_eval_df.zip"
test_pred_df_path = exp_dir / "cyber_attrs_eval_by_model_df.zip"

#### utils

In [ ]:
def count_names(df: pd.DataFrame, col_name: str):
    names = df[col_name]
    try:
        names = names.apply(eval)
    except TypeError:
        pass
    counter = Counter([x for _list in names for x in _list])
    return counter


def get_lower_names(counter: Counter):
    return {name.lower() for name in counter.keys()}

# Evaluate on test-set

In [ ]:
gold_df = pd.read_csv(test_true_df_path)
cand_df = pd.read_csv(test_pred_df_path)

In [ ]:
entity_type = "group_name"

gold_counter = count_names(gold_df, entity_type)
gold_names = get_lower_names(gold_counter)

cand_counter = count_names(cand_df, entity_type)
cand_names = get_lower_names(cand_counter)

all_names = cand_names.union(gold_names)

print(f"{entity_type}: {len(gold_names)=}")
print(f"{entity_type}: {len(cand_names)=}")
print(f"{entity_type}: {len(gold_names-cand_names)=}")
print(f"{entity_type}: {len(cand_names-gold_names)=}")
print(f"{entity_type} {len(all_names)=}")


In [ ]:
# slice into interesting lines only -> than create a column for each name the dataset was labeled by
assert gold_df.shape == cand_df.shape
gold_names_series = gold_df.copy()
gold_names_series = gold_names_series[
    (gold_df[entity_type] != "[]") |
    (cand_df[entity_type] != "[]")
    ]
gold_names_series = gold_names_series[entity_type]

cand_names_series = cand_df.copy()
cand_names_series = cand_names_series[
    (gold_df[entity_type] != "[]") |
    (cand_df[entity_type] != "[]")
    ]
cand_names_series = cand_names_series[entity_type]

comp_gold_cand_df = pd.concat([gold_names_series, cand_names_series], axis=1)
comp_gold_cand_df.columns = ["gold_names", "cand_names"]
print(f"{comp_gold_cand_df.shape=}")


# convert to be list based
for col in comp_gold_cand_df.columns:
    try:
        comp_gold_cand_df[col] = comp_gold_cand_df[col].apply(eval)
    except TypeError:
        pass

In [ ]:
prf_names = {name: PRFScore() for name in all_names}
for idx in range(comp_gold_cand_df.shape[0]):
    line_gold_names = comp_gold_cand_df.iloc[idx][0]
    line_cand_names = comp_gold_cand_df.iloc[idx][1]
    tp_name = 0

    # Calc TruePositive + FalsePositive
    for name in line_cand_names:
        if name in line_gold_names:
            prf_names[name].tp += 1
        else:
            prf_names[name].fp += 1
    # Calc FalseNegative
    for name in set(line_gold_names) - set(line_cand_names):
        prf_names[name].fn += 1


prf_dict = {name: [prf.tp, prf.fp, prf.fn,prf.recall, prf.precision, prf.fscore, (prf.tp/(prf.tp+prf.fn+1e-100))] for name, prf in prf_names.items()}
prd_df = pd.DataFrame(prf_dict).transpose()
prd_df.columns = ["tp", "fp", "fn", "recall", "precision", "f1", "acc"]
print(f"{prd_df.shape=}")
with pd.option_context('display.max_rows', 500, 'display.max_columns', 10):
    display(prd_df)

In [ ]:
prf = PRFScore()
prf.tp = prd_df.tp.sum()
prf.fp = prd_df.fp.sum()
prf.fn = prd_df.fn.sum()

prf_tot_df = pd.DataFrame([prf.tp, prf.fp, prf.fn,prf.recall, prf.precision, prf.fscore, (prf.tp/(prf.tp+prf.fn+1e-100))]).transpose()
prf_tot_df.columns = ["tp", "fp", "fn", "recall", "precision", "f1", "acc"]
prf_tot_df

# Evaluate on train-set (Copy paste the code from above)

In [ ]:
gold_df = pd.read_csv(train_true_df_path)
cand_df = pd.read_csv(train_pred_df_path)

In [ ]:
entity_type = "group_name"

gold_counter = count_names(gold_df, entity_type)
gold_names = get_lower_names(gold_counter)

cand_counter = count_names(cand_df, entity_type)
cand_names = get_lower_names(cand_counter)

all_names = cand_names.union(gold_names)

print(f"{entity_type}: {len(gold_names)=}")
print(f"{entity_type}: {len(cand_names)=}")
print(f"{entity_type}: {len(gold_names-cand_names)=}")
print(f"{entity_type}: {len(cand_names-gold_names)=}")
print(f"{entity_type} {len(all_names)=}")
################################################################################3
# slice into interesting lines only -> than create a column for each name the dataset was labeled by
assert gold_df.shape == cand_df.shape
gold_names_series = gold_df.copy()
gold_names_series = gold_names_series[
    (gold_df[entity_type] != "[]") |
    (cand_df[entity_type] != "[]")
    ]
gold_names_series = gold_names_series[entity_type]

cand_names_series = cand_df.copy()
cand_names_series = cand_names_series[
    (gold_df[entity_type] != "[]") |
    (cand_df[entity_type] != "[]")
    ]
cand_names_series = cand_names_series[entity_type]

comp_gold_cand_df = pd.concat([gold_names_series, cand_names_series], axis=1)
comp_gold_cand_df.columns = ["gold_names", "cand_names"]
print(f"{comp_gold_cand_df.shape=}")


# convert to be list based
for col in comp_gold_cand_df.columns:
    try:
        comp_gold_cand_df[col] = comp_gold_cand_df[col].apply(eval)
    except TypeError:
        pass
################################################################################
prf_names = {name: PRFScore() for name in all_names}
for idx in range(comp_gold_cand_df.shape[0]):
    line_gold_names = comp_gold_cand_df.iloc[idx][0]
    line_cand_names = comp_gold_cand_df.iloc[idx][1]
    tp_name = 0

    # Calc TruePositive + FalsePositive
    for name in line_cand_names:
        if name in line_gold_names:
            prf_names[name].tp += 1
        else:
            prf_names[name].fp += 1
    # Calc FalseNegative
    for name in set(line_gold_names) - set(line_cand_names):
        prf_names[name].fn += 1


prf_dict = {name: [prf.tp, prf.fp, prf.fn,prf.recall, prf.precision, prf.fscore, (prf.tp/(prf.tp+prf.fn+1e-100))] for name, prf in prf_names.items()}
prd_df = pd.DataFrame(prf_dict).transpose()
prd_df.columns = ["tp", "fp", "fn", "recall", "precision", "f1", "acc"]
print(f"{prd_df.shape=}")
with pd.option_context('display.max_rows', 500, 'display.max_columns', 10):
    display(prd_df)

In [ ]:
prf = PRFScore()
prf.tp = prd_df.tp.sum()
prf.fp = prd_df.fp.sum()
prf.fn = prd_df.fn.sum()

prf_tot_df = pd.DataFrame([prf.tp, prf.fp, prf.fn,prf.recall, prf.precision, prf.fscore, (prf.tp/(prf.tp+prf.fn+1e-100))]).transpose()
prf_tot_df.columns = ["tp", "fp", "fn", "recall", "precision", "f1", "acc"]
prf_tot_df